# Semi-Unsupervised Sentence Classification

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import nltk
import torch
import pandas as pd

import numpy as np
from sklearn.cluster import KMeans

!pip install nmslib 
import nmslib

nltk.download('punkt')



     |████████████████████████████████| 13.0MB 2.5MB/s 
     |████████████████████████████████| 153kB 48.2MB/s 
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
! mkdir NLP
%cd NLP

# !mkdir GloVe
# !curl -Lo GloVe/glove.840B.300d.zip http://nlp.stanford.edu/data/glove.840B.300d.zip
# !unzip GloVe/glove.840B.300d.zip -d GloVe/

!mkdir fastText
!curl -Lo fastText/crawl-300d-2M.vec.zip https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip
!unzip fastText/crawl-300d-2M.vec.zip -d fastText/

/content/NLP
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1453M  100 1453M    0     0  10.8M      0  0:02:13  0:02:13 --:--:-- 10.7M
Archive:  fastText/crawl-300d-2M.vec.zip
  inflating: fastText/crawl-300d-2M.vec  


In [ ]:
!mkdir encoder
# !curl -Lo encoder/infersent1.pkl https://dl.fbaipublicfiles.com/infersent/infersent1.pkl
!curl -Lo encoder/infersent2.pkl https://dl.fbaipublicfiles.com/infersent/infersent2.pkl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  146M  100  146M    0     0  10.9M      0  0:00:13  0:00:13 --:--:-- 12.8M


In [ ]:
! git clone https://github.com/facebookresearch/InferSent.git

from InferSent.models import InferSent


Cloning into 'InferSent'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 259 (delta 7), reused 13 (delta 4), pack-reused 240
Receiving objects: 100% (259/259), 448.95 KiB | 632.00 KiB/s, done.
Resolving deltas: 100% (131/131), done.


In [ ]:
sentences = [ 
  'Develop high-quality PHP/Python applications (web & mobile) using Magento solutions in order to support business for our Enterprise and SMB Clients.',
  'Create, and modify websites CSS, jQuery, using Magento',
  'Review internal custom code that integrates Magento with third party systems like ERP, Shipping, Inventory',
 'You will work closely with the CTO as part of the core team to roadmap and leverage appropriate technologies to deliver the business strategy.',
  'In this role, you will be responsible for the roadmap of technical development and features.',
  'Your role may include:',
  '• Research, prototyping, design, architecture, agile implementation, and/or leading development team.',
  '• Design system architecture and develop robust backend and systems',
  'The ideal candidate has:',
  '• Strong knowledge of JavaScript and contemporary JavaScript APIs such as React.JS, JQuery, Python, Django, and/or Angular.',
  '• Experience with design and development of highly scalable distributed systems',
  '• Ability to work collaboratively in teams and develop meaningful relationships to achieve common goals.',
  '• Knowledge of existing and emerging technologies',
  '• Proven project management skills',
  '• Technical Evangelist',
  'Nice to have:',
  '• Prior experience in startups, but most of all, you should have a startup mentality',
  '• Willingness to wear many hats and a strong desire to build the business from ground up',
  'What we offer:',
  '• Opportunity to build an exciting, new and unique ecosystem with almost unlimited scalability',
  '• Opportunity to have a big impact on the vision and the company',
  '• Great team',
  '• Flexible hours (40+hrs a week)',
  'Apply now',
 'Are you interested in an amazing opportunity that also makes a difference?  We help 100,000+ teachers and students learn more every school day.',
  'To do this, we have a node stack (React, GraphQL, Apollo Client, Typescript, webpack, mongoDB, Kubernetes).',
  ' ',
  'If you are great with Javascript, that\'s all we require.',
  'We want you to be opinionated, factual, but willing to compromise. You should be comfortable both giving and receiving feedback from peers. We want to make each other better.',
  'You will play a key role in carving out a future for our web application and for the future of education.',
  'With your input, help and expertise we will build an application that will ultimately collect more classroom data on student learning than any in human history.  We want to use that data to help teachers, schools, parents, students and entire communities have unprecedented academic growth.',
  '# Bonus Points If',
  '* You\'ve used React and Typescript',
  '* GraphQL makes you smile',
  '* You have implemented E2E tests on a continuous integration server',
  '* You have implemented real-time reactivity in a project using WebSockets',
  '* You like board games',
 'Working at HappyFunCorp means working with great people on an interesting range of projects. Our HQ is in Brooklyn but our team spans the globe. We’re looking for folks who are friendly, hard-working, curious, and whip smart.',
  'We are currently looking for a experienced Wordpress developer who can make clean custom plugins and modules within the Digital media sector. Experienced with PHP is essential as the project scope is backend development. ',
  'What we look for is someone with a great attitude, experience with turning ideas into fully-fledged products. If this profile sounds like you, send us through your application!',
 'POSITION SUMMARY AND PRIMARY RESPONSIBILITIES',
  'We are looking for a Python developer to rapidly build and deploy features to support highly dynamic organizations. Your primary focus will be engaged in the development of backend systems, APIs, and and working within the Amazon Web Services (AWS) framework.',
  'RESPONSIBILITIES:',
  'Design, build, and maintain efficient, reusable, and reliable Python code',
  'Integration of data storage solutions to include both SQL and NoSQL.',
  'Integration of user-facing elements developed by a rockstar group of front-end developers and designers.',
  'Identify bottlenecks and bugs, and crush them',
  'Help maintain code quality, organization and automatization',
  'Develop administrative backend systems to support customer operations.',
  'ESSENTIAL SKILLS AND EXPERIENCE',
  '5 years experience with Python',
  'Strong experience designing and implementing APIs.',
  'Experience securing hosts to include ssl, firewalls, and setting ssh keys',
  'Familiarity with concepts of MVC, ORM, and SQL queries',
  'Solid grasp on object-oriented programming',
  'Basic understanding of front-end technologies, such as JavaScript, HTML5, and CSS3',
  'A knack for writing clean, readable code',
  'Experience with Linux servers and Amazon Web Services (AWS).',
  'Understanding of fundamental design principles behind a scalable application (oCaml a plus)',
  'Strong working knowledge of a CI tool (like Jenkins) for automated deployment and testing',
  ' ',
  'DESIRED EXPERIENCE AND SKILLS',
  'Meaningful industry rapport',
  'Demonstrated track record of success',
  'Published articles or open source projects demonstrating command of your profession',
  'PERSONAL ATTRIBUTES',
  'Highly respected and well connected.',
  'Extremely detail-oriented and exhibits outstanding judgment and professionalism.',
  'Thrives in a fast-paced and entrepreneurial environment.',
  'Resourceful; work independently in an entrepreneurial environment.',
  'Adaptability to thrive in both individual and team environments; highly developed “EQ.”',
  'Is eager to learn about new technologies.',
  'Voracious reader of current software engineering and technical articles and books',
  'REQUIRED EDUCATION',
  'Bachelors in Computer Science or higher or related field.',
  'Job Type: Full-time',
  'Location: Hampstead, MD',
  'Salary: Commensurate with experience',
  'Application Questions',
  'How many years of python experience do you have?',
  'Have you completed the following level of education: Bachelor\'s?',
  'Are you within a 25 mile radius of Baltimore, MD?',
  'Are you willing to undergo a background check, in accordance with local law/regulations?',
  'Are you able to commute to this job\'s location?',
  'We only accept US citizens for this job. Are you a US citizen?',
 'Kitsuma is seeking an experienced serverless full stack engineer to join the product development team. The position is full-time and is based in Charlotte, North Carolina, although full-time working remotely is an option. ',
  'The programmer will own the code, test, build and release infrastructure for the application. This will include designing and implementing processes and tools to support continuous integration, testing, deployment, automation. In addition, the programmer will be an evangelist for good software engineering practices.',
 'Help us build an amazing EdTech platform focused on helping AP students study efficiently for their exams.',
  'You\'ll be working on a multitude of exciting features and will need to wear a lot of hats.',
  'This position will be contracted month-by-month with the opportunity to upgrade to an annual contract in the future.',
  'Feel free to send me an email if you would like more information.',
 'Sana Benefits is a tech-forward employee benefits startup seeking experienced software engineers to join our team. As a successful candidate, you have demonstrated the ability to build, deploy and maintain large-scale, distributed applications with complex database schemas. You understand and use automated testing and know how to write clean, readable code.',
  'We are building a distributed engineering team and encourage all applicants to apply, regardless of location.',
  'We work primarily in Ruby on Rails and JavaScript, but are open to those with differing backgrounds. You should have a deep understanding of how to build software that goes beyond the ability to use these tools.',
  'On the job you will:',
  '- Automate insurance claims processing',
  '- Build internal tools to support our operations team',
  '- Enhance our testing, monitoring and continuous deployment infrastructure',
  '- Help keep sensitive medical data safe and secure',
  '- Work with our operations team to build out a roadmap',
  'About you:',
  '- Minimum of 2 years of professional, hands-on experience',
  '- Strong object-oriented design and coding skills',
  '- Working knowledge of Ruby, Javascript',
  '- Experience working with complicated systems at scale',
  '- Solid understanding of data structures and algorithms',
  'Tech We Use:',
  'Rails, React, Docker, Elasticsearch, AWS, Linux, Postgres, Git',
 'We\'re looking for a cloud software developer who can build cloud backend services running on Serverless services on AWS and Google Cloud. ',
  'Our cloud software developers spend their days:',
  ' - Developing, testing and deploying cloud applications',
  ' - Delivering code in automated build, integration, testing and deployment environments',
  ' - Contributing to agile management in stand-ups and retrospectives',
  ' - Collaborating with client teams and software architects across multiple projects.',
  ' - Coaching and mentoring, pair programming and jointly developing with client teams.',
  ' - Leading development of project workstreams, mentoring less experienced developers.',
  ' - Being part of a technical community, supporting our team on company-wide projects.',
  'Candidate Requirements - you’re perfect for this position if you have:',
  ' - Experience building modern Internet and enterprise software applications.',
  ' - Led software development work streams inside larger projects.',
  ' - Proficiency developing in multiple languages - JavaScript/Node.js, Python, Java and Go.',
  ' - Automated test-driven approach to software development.',
  ' - Experience of working in CI/CD deployments with Github and AWS deployment tools.',
  ' - Comfortable working with Agile, Lean and Pair Programming styles.',
  ' - Experience with API Gateways, Functions, NoSQL databases, notifications and message queues.',
  ' - Experience with GraphQL and REST API development',
  ' - Lead the design of distributed systems',
  ' - Accessing data across multiple datastores - SQL, NoSQL and graph databases',
  ' - AWS associate developer or solutions architect certification (not required)',
  'Compensation',
  ' - A competitive salary.',
  ' - Annual bonus based upon company and individual achievements.',
  ' - 15 days of annual vacation allowance, plus 8 public holiday days PTO.',
  ' - Medical, dental and vision insurance.',
  ' - Mobile phone and home Internet allowances.',
  ' - Professional training, conferences and certifications.',
 '■ WHAT WE DO ',
  'We\'ve built behavior analysis AI which detect & predict human behaviors. With the technology, we developed “VAAKEYE” to prevent crimes before it happen, and developed a checkout-less system “VAAKPAY” to reduce operations & personnels. We stay focus on social issues.',
  '- MEDIA ',
  'Bloomberg, Forbes, etc. published in large numbers domestically and abroad ',
  'https://bloom.bg/2GWEM45',
  '■ MEMBER',
  'Technical team is comprised of researchers from international organizations, engineers from NEC, CTO experienced people (6 people) and other technically talented global members, and Biz is comprised of members such as representatives of listed companies and consultants. It has been.',
  '■ COMMENT',
  'Jerome Williams (a professor at Rutgers University) - It\'s a good approach. You shouldn\'t really profile. You should behaviorally profile.',
  '■ ABOUT OUR OFFERING',
  '[ VAAK Technology 3 Division ] ',
  '1 Behavioral analysis common base "VAAKBASE"',
  '2 Crime prevention AI "VAAKEYE" (domestic / overseas) ',
  '3 Unmanned cash register "VAAKPAY"',
  '[ Recruited Position ] ',
  '- Software Engineer',
  '- R&D Engineer',
  '[ Major areas of work ]',
  '- architect and coding of base system and products',
  '- Engineering (1+ from the followings):',
  '--- VAAKBASE: Behavior detection, Tracking system, API for Web UI (provide Video, Data)',
  '--- VAAKEYE(security&marketing): Detect&Alert system',
  '--- VAAKPAY: Management system for Shelf, Customer and Items.',
  '- Research: (1+ from the followings):',
  '--- Behavior detection/prediction, Face recognition, Gait recognition, Pose estimation, People Tracking, Object recognition',
  '- Team building and documentation',
  '- 2nd mission: you can do other domain jobs if you want',
  '[ Required condition A (1+ from the followings) ] ',
  '- Deep learning algorithm development for images and time series (1+ from the followings):',
  '--- behavior judgment / prediction · face recognition · gait recognition · POSE estimation · Tracking, object identification, mapping',
  '- System development (Python) * Software Engineer required ',
  '- Compiler and GPU software development ',
  '- Automation and Quality Assurance',
  '[ Required Condition B ] ',
  '- Ability to agree with VISION and VALUE',
  '- At least 2 year\'s work experience in the work area and surrounding area',
  '[ Technical Environment ] ',
  '- Langage: Python, Ruby, C ++ ',
  '- Framework: Keras, tensorflow, Rails, Bootstrap ',
  '- Versioning: Git ',
  '- Cloud: AWS , GCP ',
  '- Repository: Bitbucket ',
  '- Communication: Slack ',
  '- CI: CircleCI ',
  '- Monitoring: Datadog , NewRelic ',
  '- Error Monitoring: Sentry ',
  '- DB: MySQL, Redis',
  '[ others ]',
  '- Visa Support',
  'We are waiting for your application. At first, please feel free to apply!'    
]

In [ ]:
# Load infersent model 
model_version = 2
MODEL_PATH = "encoder/infersent%s.pkl" % model_version
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048, 'pool_type': 'max', 'dpout_model': 0.0, 'version': model_version}

model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))

W2V_PATH = 'fastText/crawl-300d-2M.vec'
model.set_w2v_path(W2V_PATH)

# generate infersent sentence embeddings
model.build_vocab(sentences, tokenize=True)
embs = model.encode(sentences, tokenize=True)

Found 897(/918) words with w2v vectors
Vocab size : 897


In [ ]:
model

InferSent(
  (enc_lstm): LSTM(300, 2048, bidirectional=True)
)

In [ ]:
!mkdir '../gdrive/My Drive/model'
!ls 'gdrive/My Drive/model'


ls: cannot access 'gdrive/My Drive/model': No such file or directory


In [ ]:
! ls 'gdrive'

ls: cannot access 'gdrive': No such file or directory


In [ ]:
torch.save(model.state_dict(), 'model.pt')

In [ ]:
!ls

encoder  fastText  InferSent  model  model.pt


In [ ]:
from google.colab import files
files.download('model.pt') 

In [ ]:
from google.colab import files
files.download('example.txt') 

In [ ]:
NTHREADS = 8
def create_index(a):
    index = nmslib.init(space='angulardist')
    index.addDataPointBatch(a)
    index.createIndex()
    return index

def get_knns(index, vecs, k=3):
    return zip(*index.knnQueryBatch(vecs, k=k, num_threads=NTHREADS))

nn_wvs = create_index(embs)
to_frame = lambda x: pd.DataFrame(np.array(x)[:,1:])
idxs, dists = map(to_frame, get_knns(nn_wvs, embs, k=3))
catted = pd.concat([idxs.stack().to_frame('idx'), dists.stack().to_frame('dist')], axis=1).reset_index().drop('level_1',1).rename(columns={'level_0': 'v1', 'idx': 'v2'})

indices, distances = np.vstack(idxs), np.vstack(dists)
n_samples_transform = embs.shape[0]
n_neighbors = 3
indptr = np.arange(0, n_samples_transform * n_neighbors + 1, n_neighbors)

In [ ]:
kmeans = KMeans(n_clusters=3)
kmeans.fit(embs)
y_km = kmeans.fit_predict(embs)

In [ ]:
first_cluster = [x for idx, x in enumerate(sentences) if y_km[idx] == 0]
for x in first_cluster: print(x)

In [ ]:
second_cluster = [x for idx, x in enumerate(sentences) if y_km[idx] == 1]
for x in second_cluster: print(x)

In [ ]:
third_cluster = [x for idx, x in enumerate(sentences) if y_km[idx] == 2]
for x in third_cluster: print(x)